In [1]:
import torch, yaml
import numpy as np
from cnn_models import ConvUNet
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.nn.modules.activation import ReLU
from torch import cat

In [38]:
class Interpolate(nn.Module):
    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode
        
    def forward(self, x):
        x = self.interp(x, size=self.size, mode=self.mode, align_corners=False)
        return x

class ConvUNetBis(nn.Module):
    def __init__(self, input_size, channels_init=16):
        super(ConvUNetBis, self).__init__()
        self.input_size = input_size
        self.channels_init = channels_init

        # encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=channels_init, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.inte1 = nn.Sequential(
          Interpolate(size=input_size//16, mode='bilinear'),
        )

        self.enc2 = nn.Sequential(
          nn.Conv2d(in_channels=channels_init, out_channels=channels_init*2, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
        )
        self.inte2 = nn.Sequential(
          Interpolate(size=input_size//32, mode='bilinear'),
        )
        
        self.enc3 = nn.Sequential(
          nn.Conv2d(in_channels=channels_init*2, out_channels=channels_init*4, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
        )
        self.inte3 = nn.Sequential(
          Interpolate(size=input_size//64, mode='bilinear'),
        )
        

        # linear layer internal
        self.lin1 = nn.Sequential(
            nn.Linear(in_features=(channels_init*4)*((input_size//64)**2), out_features=(channels_init*4)*((input_size//64)**2), bias=True),
            nn.ReLU()
        )

        # decoder 
        self.intd1 = nn.Sequential(
          Interpolate(size=input_size//32, mode='bilinear')
        )
        self.dec1 = nn.Sequential(
          nn.Conv2d(in_channels=2*channels_init*4, out_channels=channels_init*2, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
        )

        self.intd2 = nn.Sequential(
          Interpolate(size=input_size//16, mode='bilinear')
        )
        self.dec2 = nn.Sequential(
          nn.Conv2d(in_channels=2*channels_init*2, out_channels=channels_init, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
        )

        self.intd3 = nn.Sequential(
          Interpolate(size=input_size, mode='bilinear')
        )
        self.dec3 = nn.Sequential(
          nn.Conv2d(in_channels=2*channels_init, out_channels=3, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
        )
        
        self.dec4 = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
        )

    def compute_next_size(self, dimension, kernel, padding=0, stride=1):
        return int((dimension + 2*padding - kernel) / stride + 1)

    def forward(self, x_input):
        # encoding
        x_enc1 = self.enc1(x_input)
        x = self.inte1(x_enc1)
        x_enc2 = self.enc2(x)
        x = self.inte2(x_enc2)
        x_enc3 = self.enc3(x)
        x = self.inte3(x_enc3)
        
        # latent space operation
        x = self.lin1(x.view(-1, self.channels_init*4*((self.input_size//64)**2)))
        
        # decoder
        x = self.intd1(x.view(-1, self.channels_init*4, self.input_size//64, self.input_size//64))
        x = self.dec1(cat([x_enc3, x], dim=1))
        x = self.intd2(x)
        x = self.dec2(cat([x_enc2, x], dim=1))
        x = self.intd3(x)
        x = self.dec3(cat([x_enc1, x], dim=1))
        reconstruction = self.dec4(x)
        return reconstruction

In [48]:
class SmallLinear(torch.nn.Module):

    def __init__(self):
        super(SmallLinear, self).__init__()
        self.lin1 = torch.nn.Linear(22, 15)
        self.lin2 = torch.nn.Linear(15, 10)
        self.lin3 = torch.nn.Linear(10, 3)
        self.activation = torch.nn.Tanh()

    def forward(self, x):
        x = self.lin1(x)
        x = self.activation(x)
        x = self.lin2(x)
        x = self.activation(x)
        x = self.lin3(x)
        return x

smallLinear = SmallLinear()

### Set device

In [2]:
# setting device and data types
dtype = torch.float32 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Configuration

In [3]:
# loading the model config
with open('training_config.yml') as file:
    config = yaml.safe_load(file)
# image size
reshape_size = config['reshape_size']
scale_factor = config['scale_factor']
file_name = 'channel_x1_256_y1_512_z1_256_step2'

### Load data

In [4]:
# load data
x_all = np.load('data/' + file_name + '/xs.npy')
y_all = np.load('data/' + file_name + '/ys.npy')
u_all = np.load('data/' + file_name + '/us.npy')
v_all = np.load('data/' + file_name + '/vs.npy')
p_all = np.load('data/' + file_name + '/ps.npy')

### Create segmentation masks

In [5]:
# segmentation maps
seg_map_all = 2*np.ones_like(x_all)
seg_map_all[:, 0, :] = 1
seg_map_all[:, -1, :] = 1

### Create datasets

In [6]:
len_train = int(0.8*len(x_all))

x, y, seg_map = [
    torch.tensor(arr[:len_train, :, :], requires_grad=True).unsqueeze(-1).permute((0, 3, 1, 2)) for arr in [x_all, y_all, seg_map_all]
]

u, v, p = [
    torch.tensor(arr[:len_train, :, :], requires_grad=True).unsqueeze(-1) for arr in [u_all, v_all, p_all]
]

x_val, y_val, seg_map_val = [
    torch.tensor(arr[len_train:, :, :], requires_grad=False).unsqueeze(-1).permute((0, 3, 1, 2)) for arr in [x_all, y_all, seg_map_all]
]

u_val, v_val, p_val = [
    torch.tensor(arr[len_train:, :, :], requires_grad=False).unsqueeze(-1) for arr in [u_all, v_all, p_all]
]

# data
train_data = torch.cat([u, v, p], axis=-1)
train_data = train_data.permute((0, 3, 1, 2))
val_data = torch.cat([u_val, v_val, p_val], axis=-1)
val_data = val_data.permute((0, 3, 1, 2)) 

In [7]:
## Custom dataset
class ChannelFlowFull(Dataset):
    def __init__(self, x, y, vels, seg_map):
        super(ChannelFlowFull, self).__init__()
        self.x = x
        self.y = y
        self.seg_map = seg_map
        self.vels = vels
        return
    
    def __len__(self):
        return len(self.vels)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx], self.vels[idx], self.seg_map[idx]

In [8]:
# datasets
train_dataset = ChannelFlowFull(x, y, train_data, seg_map)
val_dataset = ChannelFlowFull(x_val, y_val, val_data, seg_map_val)

# dataloaders
batch_size = config['batch_size']
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [9]:
for x_sample, y_sample, flow_sample, seg_map_sample in train_dataloader:
    x_sample = x_sample.to(device=device, dtype=dtype)
    y_sample = y_sample.to(device=device, dtype=dtype)
    flow_sample = flow_sample.to(device=device, dtype=dtype) # move to device, e.g. GPU
    seg_map_sample = seg_map_sample.to(device=device, dtype=dtype).squeeze()
    break

### Set up model

In [58]:
model = ConvUNetBis(input_size=256, channels_init=4)
model = model.to(device=device)
model.train()
print('There are ', sum(p.numel() for p in model.parameters()), ' parameters to train.')

There are  71039  parameters to train.


In [74]:
all_params = list(model.parameters()) + list(smallLinear.parameters())

In [75]:
# Train the model
optimizer = torch.optim.Adam(all_params, lr=0.001)

In [76]:
latent_vectors = model(flow_sample)

In [77]:
latent_vectors_perm = latent_vectors.permute(0, 2, 3, 1)
interior_points = latent_vectors_perm[seg_map_sample==2] # all interior points from the batch
boundary_points = latent_vectors_perm[seg_map_sample==1] # all boundary points from the batch

In [78]:
x_sample_perm = x_sample.permute(0, 2, 3, 1)
y_sample_perm = y_sample.permute(0, 2, 3, 1)

x_interior_points = x_sample_perm[seg_map_sample==2].view(-1, 1)
x_boundary_points = x_sample_perm[seg_map_sample==1].view(-1, 1)
y_interior_points = y_sample_perm[seg_map_sample==2].view(-1, 1)
y_boundary_points = y_sample_perm[seg_map_sample==1].view(-1, 1)

In [79]:
input_features = torch.cat([x_interior_points, y_interior_points, interior_points], dim=1)
inputs = [input_features[..., i:i+1] for i in range(input_features.shape[-1])]
x_ = torch.cat(inputs, axis=-1)

In [80]:
x_.shape

torch.Size([520192, 22])

In [81]:
print('There are ', sum(p.numel() for p in smallLinear.parameters()), ' parameters to train.')

There are  538  parameters to train.


In [82]:
outputs_linear = smallLinear(x_)

In [83]:
dudx, dudy = torch.autograd.grad(outputs_linear[:,0], [inputs[0], inputs[1]], grad_outputs=torch.ones_like(outputs_linear[:, 0]).view(-1), retain_graph=True, create_graph=True)
d2udx2 = torch.autograd.grad(dudx, inputs[0], grad_outputs=torch.ones_like(dudx), retain_graph=True, create_graph=True)
d2udy2 = torch.autograd.grad(dudy, inputs[1], grad_outputs=torch.ones_like(dudy), retain_graph=True, create_graph=True)

In [84]:
loss = torch.mean((dudx + dudy)**2)

In [85]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [86]:
loss

tensor(0.0006, grad_fn=<MeanBackward0>)